# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![Map/shuffle/reduce](https://image.slidesharecdn.com/datasciencewebinar061312-120618165932-phpapp02/95/how-salesforcecom-uses-hadoop-41-728.jpg?cb=1340920667)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

`map` is a Higher Order Function (HOF) that takes a function f and a sequence and returns a new sequence formed by applying f to each element in the original sequence. 

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

Very often we write lambdas, anonymous functions, to use in defining simple transformations with `map`:


In [7]:
map

map

In [8]:
sequence = [7, 8, 11, 2]

lambda x: '' + x

<function __main__.<lambda>(x)>

In [9]:
def saludito(person):
    return 'holi ' + person
saludito('pepe')

'holi pepe'

In [10]:
list(map(lambda x: x * x, sequence))

[49, 64, 121, 4]

In [11]:
def square(n):
    return n * n

In [12]:
list(map(square, sequence))

[49, 64, 121, 4]

In [13]:
sequence = [7, 8, 11, 2]
result = []
for element in sequence:
    result.append(element ** 2)
result

[49, 64, 121, 4]

We can also use named functions, of course. Note that we pass them as arguments to `map`, not execute them. You can think of them as tools we give to `map` for it to use.

Consider the following very common pattern: create empty list, append transformed elements one by one:

We can abstract over every possible list by defining a function that takes the list as an argument:

In [14]:
sequence = ['coche', 'moto', 'furgo']

def plural(sequence):
    
    result = []
    for element in sequence:
        result.append(element + 's')
        
    return result
plural(sequence)

['coches', 'motos', 'furgos']

In [15]:
plural(['naranja', 'pera'])

['naranjas', 'peras']

Consider this other piece of code: we are doing a very similar thing, but with a different function. A lot of the code is common between the two little programs.

Wouldn't it be convenient if we could abstract over every function, as we did over every list?

In [16]:
sequence = ['coche', 'moto', 'furgo']


result = []
for element in sequence:
    result.append(element[0])
        
result


['c', 'm', 'f']

We can, taking the function to be applied as an argument. This is because Python treats functions as first-class objects, so they can be used as arguments to other functions.

In [17]:
sequence = ['coche', 'moto', 'furgo']

def plural(f, sequence):
    
    result = []
    for element in sequence:
        result.append(f(element))
        
    return result

plural(lambda x: x[0],sequence)

['c', 'm', 'f']

That function we just wrote, `process_list`, is exactly what `map` is

In [18]:
list(map(lambda n:n + 's', sequence))

['coches', 'motos', 'furgos']

**Filtering**

`filter` encodes another very common pattern: building a sequence from some of the elements of an input sequence, deciding whether to include each based on the result of evaluating a function, often called the `predicate`, on each of the elements.


![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [19]:
filter?

In [20]:
number = [2,4,6,7,12,15,40]

In [21]:
list(filter(lambda x: x< 10, number))

[2, 4, 6, 7]

Again, we can use either lambdas or named functions.

#### Exercise

write a filter to extract those words that have more than 3 letters from `collection_of_strings`

In [22]:
collection_of_strings = ['holi','que','tal','te', 'va', 'a', 'ti', 'guapi']
list(filter(lambda x: len(x) > 3, collection_of_strings))

['holi', 'guapi']

#### Exercise

Write a filter to extract prime numbers from the following list of integers:

```python
[12, 17, 19, 18, 23, 24]
```

You will need to write a function that determines whether a single number is a prime or not.

In [23]:
def prime(n):
    for v in range(2, n -1):
        if n % v == 0:
            return False
    return True

In [24]:
numbers = [12, 17, 19, 18, 23, 24]
list(filter(prime, numbers))

[17, 19, 23]

`map` and `filter` are not  used explicitly very often in Python, in part because of their cumbersome syntax. 

Nevertheless, there is a well-known feature in Python that is in fact just a convenient way to write `map`s and `filter`s: **list comprehensions**:

In [25]:
[ i *  i for i in numbers]

[144, 289, 361, 324, 529, 576]

In [26]:
[ i *  i for i in numbers if prime(i)]

[289, 361, 529]

The above list comprehension is exactly equivalent to:

In [27]:
list(map(lambda x: x*x, filter(prime, numbers)))

[289, 361, 529]

Which is a pipeline of two steps: first a `filter`, then a `map`

**Reducing** 

`reduce` is the third basic foundation of functional programming. Reduce uses a function, often called the `combiner`, to transform a sequence of elements of type T (that is, any type) into a single T. The combiner must take 2 Ts and return only one T.


Recall it must be commutative! Think about the importance of this when parallelizing computations

![reduce](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [28]:
from functools import reduce
number = [2, 3, 5, 7]
reduce(lambda x, y: x+y, number)

17

#### Exercise

Write a function_x such that, when applied with `reduce`, that reduce returns the highest number in a list. Don't use the built-in `max`!

In [29]:
numbers = [12, 17, 19, 18, 23, 24]
def x(a, b):
    if a > b:
        return a
    return b

In [30]:
reduce(x, numbers)

24

## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in mehots sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

#### First part

* Do a reduce to do the sum, and a different map-reduce to get the length

In [31]:
sum = reduce(lambda a, b: a + b, numbers) 

That was also a pipeline of two steps. It's equivalent to:

In [32]:
elementsList = list(map(lambda x: 1, numbers))
elementsList

[1, 1, 1, 1, 1, 1]

In [33]:
length =  reduce(lambda a, b: a+b, elementsList)
length

6

In [34]:
sum / length

18.833333333333332

#### Combine them in one pass

Think about what it would mean to finish one computation and only then do another in a distributed environment.

Combine both in one pass:

In [35]:
def mean(values):
    sum = reduce(lambda a, b: a + b, numbers) 
    elementsList = list(map(lambda x: 1, values))
    length =  reduce(lambda a, b: a+b, elementsList)
    return sum / lengthArray(values)

In [36]:
def lengthArray(values):
    elementsList = list(map(lambda x: 1, values))
    length =  reduce(lambda a, b: a+b, elementsList)
    return length

In [37]:
mean(numbers)

18.833333333333332

In [38]:
total, count = reduce(lambda a, b: (a[0] + b[0], a[1] + b[1]), map(lambda n: (n, 1), numbers))
total / count

18.833333333333332

## (1c) Exercise: Calculate the standard deviation of a collection of real numbers

We'll use the typical definition of the standard deviation:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1}$$

For this, use the *mean* and *count* variables from the previous excercise.

In [39]:
m = total / count
q = list(map(lambda x: (x - m)**2, numbers))
reduce(lambda a,b: a + b, q) / (count - 1)


18.96666666666667

In standard idiomatic Python we'd do it like this:

With `map` and `reduce`:

#### Calculate the standard deviation in one pass.

Again, think about what it would mean to finish one computation and only then do another in a distributed environment.


If we can, we should avoid broadcasting variables all over the cluster. In this case, the following alternative definition of standard deviation can come in handy:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$



In [40]:
squares, total, count = reduce(lambda a, b: (a[0]+b[0], a[1] + b[1], a[2] + b[2]), list(map(lambda n: (n ** 2, n, 1), numbers)))

mean = total / count
variance = (squares - count * mean ** 2) / (count -1)
variance 

18.966666666666697

#### Side note

In Python 2, map and filter returned lists. In Python 3, they return generators, which are lazy collections. They are somewhat similar to files in that they can be depleted of elements after iterating through them.

In [41]:
plurals = map (lambda word: word + 's', ['coche', 'moto'])

for word in plurals:
    print(word)

coches
motos


In [42]:
for word in plurals:
    print(word)

## (1d) The 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

We are going to use as test case a list of elements chosen from a small set:

In [43]:
import random 
random.seed(42)
mythical_creatures = random.choices(['fairy', 'elf', 'gnome', 'troll'], k=100)

In [44]:
mythical_creatures

['gnome',
 'fairy',
 'elf',
 'fairy',
 'gnome',
 'gnome',
 'troll',
 'fairy',
 'elf',
 'fairy',
 'fairy',
 'gnome',
 'fairy',
 'fairy',
 'gnome',
 'gnome',
 'fairy',
 'gnome',
 'troll',
 'fairy',
 'troll',
 'gnome',
 'elf',
 'fairy',
 'troll',
 'elf',
 'fairy',
 'fairy',
 'troll',
 'gnome',
 'troll',
 'gnome',
 'gnome',
 'troll',
 'elf',
 'gnome',
 'troll',
 'gnome',
 'troll',
 'gnome',
 'gnome',
 'fairy',
 'fairy',
 'elf',
 'fairy',
 'fairy',
 'fairy',
 'elf',
 'gnome',
 'elf',
 'elf',
 'fairy',
 'elf',
 'troll',
 'gnome',
 'gnome',
 'fairy',
 'gnome',
 'fairy',
 'elf',
 'troll',
 'gnome',
 'gnome',
 'gnome',
 'troll',
 'troll',
 'fairy',
 'fairy',
 'elf',
 'elf',
 'fairy',
 'troll',
 'troll',
 'elf',
 'gnome',
 'elf',
 'troll',
 'elf',
 'elf',
 'fairy',
 'gnome',
 'elf',
 'gnome',
 'troll',
 'elf',
 'fairy',
 'troll',
 'gnome',
 'fairy',
 'fairy',
 'fairy',
 'gnome',
 'troll',
 'elf',
 'fairy',
 'elf',
 'troll',
 'gnome',
 'troll',
 'troll']

### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [45]:
result = {}
for creature in mythical_creatures:
    if creature not in result:
        result[creature] = 1
    else:
        result[creature] += 1
        
result

{'gnome': 28, 'fairy': 29, 'elf': 21, 'troll': 22}

### (1d.2) Map/reduce

* Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, the first thing to do is to map our collection to the type of the output. 
 
* The final result will be a dictionary of words in the vocabulary and counts. Therefore, we need to map each input word to this type.
 
* Then, we have to define a function, the combiner, that combines these dictionaries two at a time.

In [46]:
list(map(lambda creature: {creature: 1}, mythical_creatures))


[{'gnome': 1},
 {'fairy': 1},
 {'elf': 1},
 {'fairy': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'troll': 1},
 {'fairy': 1},
 {'elf': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'gnome': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'fairy': 1},
 {'gnome': 1},
 {'troll': 1},
 {'fairy': 1},
 {'troll': 1},
 {'gnome': 1},
 {'elf': 1},
 {'fairy': 1},
 {'troll': 1},
 {'elf': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'troll': 1},
 {'gnome': 1},
 {'troll': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'troll': 1},
 {'elf': 1},
 {'gnome': 1},
 {'troll': 1},
 {'gnome': 1},
 {'troll': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'elf': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'fairy': 1},
 {'elf': 1},
 {'gnome': 1},
 {'elf': 1},
 {'elf': 1},
 {'fairy': 1},
 {'elf': 1},
 {'troll': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'fairy': 1},
 {'gnome': 1},
 {'fairy': 1},
 {'elf': 1},
 {'troll': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'gnome': 1},
 {'troll': 1},
 {'troll': 1},
 {'fairy': 1},
 {'fairy': 1},
 {

In [47]:
def dict_combiner(left, right):
    keys = []
    for k1 in left.keys():
      keys.append(k1)
    for k1 in right.keys():
      keys.append(k1) 

    result = {}
    
    for key in keys:
        result[key] = left.get(key, 0) + right.get(key, 0)
    
    return result


dict_combiner({'f':1, 'g':3},{'tttt':1})            

{'f': 1, 'g': 3, 'tttt': 1}

In [48]:
reduce(dict_combiner, map(lambda creature: {creature: 1}, mythical_creatures))

{'gnome': 28, 'fairy': 29, 'elf': 21, 'troll': 22}

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its lineage), instead of the data itself. This property ensures fault-tolerance.
 * Users can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you could use learning a bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

We use sc.parallelize to convert a standard Python collection into an RDD.

In [56]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [57]:
rdd = sc.parallelize(mythical_creatures)

In Spark, `map` is a method of the RDD type.

In [58]:
plurals = rdd.map(lambda x: x + 's')
plurals

PythonRDD[7] at RDD at PythonRDD.scala:53

**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include counting, collecting, reducing, taking, etc. Take a look at the [Spark programming guide](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions)


In [59]:
plurals.take(5)

['gnomes', 'fairys', 'elfs', 'fairys', 'gnomes']

#### Exercise

Calculate, from the rdd generated from list_of_cakes, how many cakes we have whose name does **not** start with an 's'

In [60]:
rdd.filter(lambda x: x[0] != 'f' ).count()

71

A very common way to write these chained operations in Spark is to put each on a line. Due to Python's syntax, we need to escape the newline character with a backslash.

In [61]:
rdd.filter(lambda x: x[0] != 'f' )\
.count()

71

### Actions and Transformations

There are two main types of operations in Spark:
[Actions](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions) and [Transformations](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#transformations).

**Transformations** produce an RDD. Some of the most important are `map`, `filter`, and `reduceByKey`. `sc.parallelize` and `sc.textFile` are not technically Transformations but you can think of them as such in one very important respect: they are *lazyly evaluated*. That is, when we perform a Transformation, we are only describing the operation to be performed at some point in the future. They attach a node to the execution graph.

**Actions** are implemented as methods on an RDD, and return an object of a type **that is not an RDD**. When we perform an Action, we give the order to execute the previously described transformations: we trigger the execution of the graph. Some of the most important are `reduce`, `collect`, `take`, `takeOrdered`, and `count`.

In [62]:
rdd.map(lambda word: word + 5).take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 19, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipyt

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazyly evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

In [63]:
plurals.persist()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [64]:
plurals.cache()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [65]:
from pyspark import StorageLevel

In [66]:
StorageLevel.MEMORY_ONLY

StorageLevel(False, True, False, False, 1)

In [67]:
plurals.count()

100

In [68]:
plurals.unpersist()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [69]:
plurals.persist(StorageLevel.DISK_ONLY)

PythonRDD[7] at RDD at PythonRDD.scala:53

StorageLevel cannot be changed after it has been set. We need to `unpersist` first.

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methods like `repartition(), coalesce()`, etc.

In [70]:
rdd.getNumPartitions()

4

In [71]:
repa_rdd = rdd.repartition(20)
repa_rdd.getNumPartitions()

20

In [72]:
colaes_rdd = repa_rdd.coalesce(4)
colaes_rdd.getNumPartitions()

4

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it retruns an RDD created by coalescing all elements within each partition into a list.

In [73]:
colaes_rdd.glom().map(len).collect()

[50, 0, 45, 5]

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enormous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

In [80]:
pair_rdd = rdd.map(lambda creature: (creature, 1))
pair_rdd.take(4)

[('gnome', 1), ('fairy', 1), ('elf', 1), ('fairy', 1)]

### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [76]:
grouped = pair_rdd.groupByKey()
grouped.collect()

[('fairy', <pyspark.resultiterable.ResultIterable at 0x7f3d6b770860>),
 ('troll', <pyspark.resultiterable.ResultIterable at 0x7f3d6b770828>),
 ('gnome', <pyspark.resultiterable.ResultIterable at 0x7f3d6b7708d0>),
 ('elf', <pyspark.resultiterable.ResultIterable at 0x7f3d6b770940>)]

In [77]:
grouped.map(lambda tuple_: (tuple_[0], len(tuple_[1]))).collect()

[('fairy', 29), ('troll', 22), ('gnome', 28), ('elf', 21)]

In [78]:
grouped.mapValues(len).collect()

[('fairy', 29), ('troll', 22), ('gnome', 28), ('elf', 21)]

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

In [84]:
pair_rdd.reduceByKey(lambda v1, v2: v1+v2).collect()

[('fairy', 29), ('troll', 22), ('gnome', 28), ('elf', 21)]

#### Summary: word count in Spark:

In [86]:
rdd.map(lambda word: (word, 1)) \
    .reduceByKey(lambda a,b:a+b) \
    .collect()

[('fairy', 29), ('troll', 22), ('gnome', 28), ('elf', 21)]

In [87]:
from operator import add

rdd.map(lambda word: (word, 1)) \
    .reduceByKey(add) \
    .collect()

[('fairy', 29), ('troll', 22), ('gnome', 28), ('elf', 21)]

## (2d) Apply word count to a file

### ** (2d.1) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

In [89]:
!wget -v http://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt

--2019-05-04 09:54:24--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5798834 (5,5M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   5,53M   231KB/s    in 85s     

2019-05-04 09:56:07 (66,8 KB/s) - ‘shakespeare.txt’ saved [5798834/5798834]



In [92]:
lines = sc.textFile('shakespeare.txt')
lines.take(5)

['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and']

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [94]:
import re

re.sub('ö', 'oe', 'Köning')

'Koening'

In [97]:
def remove_punctutation(line):
    return re.sub('[^\\w ]', '', line)

line = 'whatsoever.  You may copy it, give it away or re-use it under the terms'
remove_punctutation(line)

'whatsoever  You may copy it give it away or reuse it under the terms'

In [101]:
clean_lines = lines.map(remove_punctutation).map(str.lower)
clean_lines.take(20)

['',
 'project gutenbergs the complete works of william shakespeare by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever  you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'wwwgutenbergorg  if you are not located in the united states youll',
 'have to check the laws of the country where you are located before using',
 'this ebook',
 '',
 'see at the end of this file  content note added in 2017 ',
 '',
 '',
 'title the complete works of william shakespeare',
 '',
 'author william shakespeare',
 '',
 'release date january 1994 ebook 100']

http://www.regular-expressions.info/quickstart.html

https://regex101.com/

### ** (2d.3) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/3/library/stdtypes.html#str.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

Check out the Spark Programming Guide for an alternative transformation that helps us here.

In [107]:
word_list = clean_lines.flatMap(str.split)

### (2d.4) Count the words and show the top 15

We know the drill at this point, don't we? We map to a tuple then `reduceByKey`

We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pair tuples, we need a custom sort function that sorts using the value part of the pair rather than the key.

You'll notice that many of the words are common English words (know as stopwords).

Use our map reduce and and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [114]:
word_list.map(lambda word: (word, 1)).reduceByKey(lambda a,b: a+b).takeOrdered(15, key=lambda o: -o[1])

[('the', 30083),
 ('and', 28378),
 ('i', 21900),
 ('to', 20796),
 ('of', 18800),
 ('a', 16085),
 ('you', 14437),
 ('my', 13180),
 ('in', 12142),
 ('that', 11782),
 ('is', 9715),
 ('not', 9064),
 ('with', 8516),
 ('me', 8263),
 ('for', 8189)]

In [117]:
sc.textFile('shakespeare.txt')\
    .map(remove_punctutation)\
    .map(str.lower)\
    .flatMap(str.split)\
    .map(lambda word: (word, 1))\
    .reduceByKey(add) \
    .takeOrdered(15, key=lambda o: -o[1])

[('the', 30083),
 ('and', 28378),
 ('i', 21900),
 ('to', 20796),
 ('of', 18800),
 ('a', 16085),
 ('you', 14437),
 ('my', 13180),
 ('in', 12142),
 ('that', 11782),
 ('is', 9715),
 ('not', 9064),
 ('with', 8516),
 ('me', 8263),
 ('for', 8189)]

# Practical

## ETL with airline coupon data

Load the data first: coupon data

In [118]:
coupon = sc.textFile('coupon150720.csv')
coupon.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

#### Exercise

Take fields 0, 2, 3, 4, and 6 from each line of coupons

In [133]:
def getValues(line):
    fields = line.split(',')
    return (fields[0], fields[2], fields[3], fields[4], float(fields[6]))

coupon_f = coupon.map(getValues)
coupon_f.take(5)

[('79062005698500', 'MAA', 'AUH', '9W', 56.79),
 ('79062005698500', 'AUH', 'CDG', '9W', 84.34),
 ('79062005924069', 'CJB', 'MAA', '9W', 60.0),
 ('79065668570385', 'DEL', 'DXB', '9W', 160.63),
 ('79065668737021', 'AUH', 'IXE', '9W', 152.46)]

#### Intermission: Stack traces in Spark

Stack traces in pyspark are quite convoluted because we have many layers of processing. Look for the python stack trace contained within the Py4JJavaError message.

#### Exercise

Keep only the amount. Get average, max, min and std

In [141]:
amount = coupon_f.map(lambda l: (l[4] ** 2, l[4],  1, l[4], l[4]))
amount.take(5)

[(3225.1041, 56.79, 1, 56.79, 56.79),
 (7113.235600000001, 84.34, 1, 84.34, 84.34),
 (3600.0, 60.0, 1, 60.0, 60.0),
 (25801.9969, 160.63, 1, 160.63, 160.63),
 (23244.051600000003, 152.46, 1, 152.46, 152.46)]

In [144]:
def tuple_combiner(tuple1, tuple2):
    return (tuple1[0] + tuple2[0],
    tuple1[1] + tuple2[1],
    tuple1[2] + tuple2[2],
    max(tuple1[3], tuple2[3]),
    min(tuple1[4], tuple2[4]))
    

In [146]:
sum_squares, total, count, max_v, min_v = amount.reduce(tuple_combiner )

In [147]:
mean = total / count
std = (sum_squares - count * mean ** 2) ** 0.5
mean, std, max_v, min_v


(149.94532037164157, 11078640.911809502, 6355194.0, 0.0)

#### All in one go!



#### Exercise

Get stats on ticket amount for all tickets with destination MAD

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

#### Part 1

We need to filter coupons with destination Madrid, and after that group on the origin. 

At this point, we only need the origin, to group on, and the value, to sum

We are now ready to reduce all amounts per origin.

#### Part 2

This is very similar, with two differences: we need to group on the airline, and to calculate averages we need to keep track of both the total amount on the coupons and the number of coupons.

Now we reduce tuples, summing each component of the tuple separately.

This reduceByKey generates an RDD of the form (k, (v1, v2)). We can map it like this:

Or we can use the mapValues method to only process the (v1, v2) part, ignoring the key for processing without discarding it.

And we're ready to take our result.

#### Stretch: 

Get the totals from first origin without using the coupon number

We'll use groupBy this time, because we need to look at every coupon in a ticket in order to identify first origin.

A ResultIterable is a lazy collection, so we can take it and iterate over it, or turn it into a list to materialize it. This is useful to get a test case to test the function we will be writing.

We are going to compare the set of origins to the set of destinations. The one that is in the first but not in the second should be the first origin.

Now we have have our values, so we are ready to get the totals and sort. Before we reduceByKey, we need to reshape our tuples so the first origin is the key.

### Further resources

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf